In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
from timeit import Timer, timeit, repeat
import casadi as cs

In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


In [3]:
def id_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    invd_j(q, qdot, qddot)
    

def id_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    invd(q, qdot, qddot)
    
    
    
def C_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_j(q, qdot)
    

def C_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C(q, qdot)
    
    
    
def g_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_j(q)
    

def M_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M(q)
    

    
def M_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_j(q)
    

def g_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g(q)
    
    

def fdaba_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_j(q, qdot, tau)
    

def fdaba_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba(q, qdot, tau)
    
    
    
def fdcrba_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_j(q, qdot, tau)
    

def fdcrba_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba(q, qdot, tau)

In [4]:
#storage for timing data
mediantime_g = 0
mediantime_gj = 0

mediantime_C = 0
mediantime_Cj = 0

mediantime_id = 0
mediantime_idj = 0

mediantime_M = 0
mediantime_Mj = 0

mediantime_fd_aba = 0
mediantime_fd_abaj = 0

mediantime_fd = 0
mediantime_fd_crbaj = 0

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/ur5_mod.urdf"
#end_link = "link" + str(urdf_nr[i])
root = 'base_link'
tip = "tool0"


robot = u2c.URDFparser()
robot.from_file(path_to_urdf)

jlist, names, qmax, qmin = robot.get_joint_info(root, tip)
njoints = robot.get_n_joints(root, tip)


#u2c and pybullet declarations
q = [None]*njoints
qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)
qdot = [None]*njoints
qddot = [None]*njoints
o = [None]*njoints
tau = [None]*njoints

gravity = [0, 0, -9.81]
C = robot.get_coriolis_rnea(root, tip)
g = robot.get_gravity_rnea(root, tip, gravity)
invd = robot.get_inverse_dynamics_rnea(root, tip, gravity)
fd_aba = robot.get_forward_dynamics_aba(root, tip, gravity)
fd_crba = robot.get_forward_dynamics_crba(root, tip, gravity)
M = robot.get_inertia_matrix_crba(root, tip)

C_j = cs.jacobian(C(qs, qsdot), qs)
g_j = cs.jacobian(g(qs), qs)
M_j = cs.jacobian(M(qs), qs)
invd_j = cs.jacobian(invd(qs, qsdot, qsddot), qs)
fd_aba_j = cs.jacobian(fd_aba(qs, qsdot, taus), qs)
fd_crba_j = cs.jacobian(fd_crba(qs, qsdot, taus), qs)


g_j = cs.Function("g_j", [qs], [g_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
C_j = cs.Function("C_j", [qs, qsdot], [C_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
M_j = cs.Function("M_j", [qs], [M_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
invd_j = cs.Function("invd_j", [qs, qsdot, qsddot], [invd_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

/home/lmjohann/urdf2casadi/examples/urdf/ur5_mod.urdf


In [5]:
n_invd = invd.n_nodes()
n_C = C.n_nodes()
n_M = M.n_nodes()
n_g = g.n_nodes()
n_fd_aba = fd_aba.n_nodes()
n_fd_crba = fd_crba.n_nodes()

n_invdj = invd_j.n_nodes()
n_Cj = C_j.n_nodes()
n_Mj = M_j.n_nodes()
n_gj = g_j.n_nodes()
n_fd_abaj = fd_aba_j.n_nodes()
n_fd_crbaj = fd_crba_j.n_nodes()

print "\nur5_nodes_id_j=", n_invdj
print "\nur5_nodes_c_d=", n_Cj
print "\nur5_nodes_m_d=", n_Mj
print "\nur5_nodes_g_d=", n_gj
print "\nur5_nodes_fdaba_d", n_fd_abaj
print "\nur5_nodes_fdcrba_d =",n_fd_crbaj

print "\nur5_nodes_id=", n_invd
print "\nur5_nodes_c=", n_C
print "\nur5_nodes_m=", n_M
print "\nur5_nodes_g=", n_g
print "\nur5_nodes_fdaba=", n_fd_aba
print "\nur5_nodes_fdcrba =", n_fd_crba     


ur5_nodes_id_j= 3069

ur5_nodes_c_d= 3006

ur5_nodes_m_d= 4095

ur5_nodes_g_d= 881

ur5_nodes_fdaba_d 10189

ur5_nodes_fdcrba_d = 15194

ur5_nodes_id= 696

ur5_nodes_c= 648

ur5_nodes_m= 1026

ur5_nodes_g= 199

ur5_nodes_fdaba= 1948

ur5_nodes_fdcrba = 2354


In [6]:
nitr = 10000
nrepeat = 100

timeit_idj = repeat("id_j_func()", setup = "from __main__ import id_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr

timeit_gj = repeat("g_j_func()", setup = "from __main__ import g_j_func", repeat = nrepeat, number = nitr)
mediantime_gj = median(timeit_gj)/nitr

timeit_Cj = repeat("C_j_func()", setup = "from __main__ import C_j_func", repeat = nrepeat, number = nitr)
mediantime_Cj = median(timeit_Cj)/nitr

timeit_Mj = repeat("M_j_func()", setup = "from __main__ import M_j_func", repeat = nrepeat, number = nitr)
mediantime_Mj = median(timeit_Mj)/nitr

timeit_fdabaj = repeat("fdaba_j_func()", setup = "from __main__ import fdaba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr

timeit_fdcrbaj = repeat("fdcrba_j_func()", setup = "from __main__ import fdcrba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr

   
print "\nur5_id_d=", mediantime_idj
print "\nur5_g_d=", mediantime_gj
print "\nur5_C_d=", mediantime_Cj
print "\nur5_M_d=", mediantime_Mj
print "\nur5_fdaba_d=", mediantime_fd_abaj
print "\nur5_fdcrba_d=", mediantime_fd_crbaj


ur5_id_d= 3.76435995102e-05

ur5_g_d= 1.57685041428e-05

ur5_C_d= 2.74915575981e-05

ur5_M_d= 2.14653491974e-05

ur5_fdaba_d= 4.36167955399e-05

ur5_fdcrba= 4.81171488762e-05


In [7]:
timeit_id = repeat("id_func()", setup = "from __main__ import id_func", repeat = nrepeat, number = nitr)
mediantime_id = median(timeit_id)/nitr

timeit_g = repeat("g_func()", setup = "from __main__ import g_func", repeat = nrepeat, number = nitr)
mediantime_g = median(timeit_g)/nitr

timeit_C = repeat("C_func()", setup = "from __main__ import C_func", repeat = nrepeat, number = nitr)
mediantime_C = median(timeit_C)/nitr

timeit_M = repeat("M_func()", setup = "from __main__ import M_func", repeat = nrepeat, number = nitr)
mediantime_M = median(timeit_M)/nitr

timeit_fdaba = repeat("fdaba_func()", setup = "from __main__ import fdaba_func", repeat = nrepeat, number = nitr)
mediantime_fd_aba = median(timeit_fdaba)/nitr

timeit_fdcrba = repeat("fdcrba_func()", setup = "from __main__ import fdcrba_func", repeat = nrepeat, number = nitr)
mediantime_fd_crba = median(timeit_fdcrba)/nitr

   
print "\nur5_id=", mediantime_id
print "\nur5_g=", mediantime_g
print "\nur5_C=", mediantime_C
print "\nur5_M= ", mediantime_M
print "\nur5_fdaba=", mediantime_fd_aba
print "\nur5_fdcrba=", mediantime_fd_crba


ur5_id= 3.64641666412e-05

ur5_g= 1.54621839523e-05

ur5_C= 2.6023542881e-05

ur5_M=  1.7515194416e-05

ur5_fdaba= 3.72484564781e-05

ur5_fdcrba= 3.97567033768e-05
